In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import glob
import os
import cv2
import sys

In [ ]:
##########################################################
# 2. LOAD DATASET
##########################################################
data = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv",header=0)# header 0 means the first row is name of the coloumn 

# Delete unused columns
data.drop(["Unnamed: 32","id"], axis=1, inplace=True)

# Change label M(ganas = malignant) = 1 dan B(jinak = benign) = 0
data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis]

# Test select malignant data
m_data = data.loc[data['diagnosis'] == 1]

# Test select benign data
b_data = data.loc[data['diagnosis'] == 0]

# View sample data
b_data.head(10) 

In [ ]:
##########################################################
# 3. SHARE TO TEST AND TRAIN DATA
##########################################################
x = data.iloc[:, 1:]
y = data['diagnosis'].tolist()

# Share test and train data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

y_test.shape

In [ ]:
print('Train data: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test data: X=%s, y=%s' % (x_test.shape, y_test.shape))

In [ ]:
x_train = x_train.values.reshape((x_train.shape[0],x_train.shape[1], 1))
x_test  = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
print('Train data: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test data: X=%s, y=%s' % (x_test.shape, y_test.shape))

In [ ]:
#CHECK SHAPE DATA
print('x train shape : ', x_train.shape)
print('y_train shape : ', y_train.shape)
print('x_test shape : ', x_test.shape)
print('y_test shape : ', y_test.shape)

In [ ]:
y_train

In [ ]:
x_train

In [ ]:
#normalize inputs:
x_train2 = x_train/255 - 0.5
x_test2 = x_test/255 - 0.5

cancer_classes = [1,0]

# Convert class labels to one-hot encoded
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)

y_train_indexes = []
for classs in y_train:
    y_train_indexes.append(cancer_classes.index(classs))
    
# convert class labels to one-hot encoded:
y_train2 = keras.utils.to_categorical(y_train_indexes)

In [ ]:
#import necessary building blocks:
from statistics import mean
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
# model
model=Sequential()
# layers
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(30,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='softmax'))

model.summary()

In [ ]:
# compiling the model
model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.005),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:

#fitting the model
history=model.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test))


In [ ]:
# Predict using testing data without labels/classes
y_pred_test = model.predict(x_test2)
y_pred_test_classes = np.argmax(y_pred_test, axis=1) # Change to normal classes
y_pred_test_classes

In [ ]:
# Create the same format for actual classes
y_actual_test_classes = np.argmax(y_test2, axis=1) # Change to normal classes
y_actual_test_classes

In [ ]:
##########################################################
# MULTI-CLASS CONFUSION MATRIX FOR EACH CLASS
##########################################################
from sklearn.metrics import multilabel_confusion_matrix
from math import sqrt

# Actual and predicted classes
lst_actual_class = y_actual_test_classes
lst_predicted_class = y_pred_test_classes

# Class = Label 0 to 1
lst_classes = [0,1]

# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]
    
    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);
    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(f1Score);
    store_mcc.append(MCC);
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(f1Score));
    print("MCC: {0}\n".format(MCC));

In [ ]:
##########################################################
# 8. OVERALL - FINAL PREDICTION PERFORMANCE
##########################################################
print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))